# Project 2：NASA Data Acquisation, Visualization, and Analysis

In [1]:
# The code makes sure that once there is change in the 'src/' folder, the 
# change will be automatically reloaded in the notebook.
%reload_ext autoreload
%autoreload 2
%aimport src

### Task 1: Understanding the NASA API and Data Collection

- Register for a NASA API key and understand the different types of data that the API provides.
- Run the Python script below to fetch data about **Near Earth Objects (NEOs)** from the NASA API for a years data.
- Extract and understand the different pieces of data provided for each NEO.

In [2]:
import requests
import time
from datetime import datetime, timedelta
from getpass import getpass

# Set your NASA API KEY, this step asks you to enter your API KEY.
# (The input box may be float in the top on your editor.)
api_key = getpass()

In [23]:
# Set the start and end dates for the data you want to fetch
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')
end_date = start_date + timedelta(days=14)  # 1 year later

# Initialize a list to store the data
data = []

# Fetch data from the NASA API 7 days at a time
# The introduction of the API is on https://api.nasa.gov, under "Browse APIs" -> "Asteroids NeoWs"
# You can look into the example query in the link below to see what the data look like:
# https://api.nasa.gov/neo/rest/v1/feed?start_date=2015-09-07&end_date=2015-09-08&api_key=DEMO_KEY
current_date = start_date
while current_date < end_date:
    next_date = min(current_date + timedelta(days=7), end_date)
    response = requests.get(f'https://api.nasa.gov/neo/rest/v1/feed?start_date={current_date.strftime("%Y-%m-%d")}&end_date={next_date.strftime("%Y-%m-%d")}&api_key={api_key}')
    data.append(response.json())
    current_date = next_date + timedelta(days=1) #Incremented current_date by a day. Necessary to prevent overlap
    time.sleep(1)  # To avoid hitting the rate limit

# Now 'data' contains the NEO data for the 1-year period

In [63]:
# Check the date coverage of your data.
dates_contained_in_data = []
for d in data:
    dates_contained_in_data += list(d['near_earth_objects'].keys())

print(sorted(dates_contained_in_data))

['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15']


In [62]:
print(data)

[{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2022-01-08&end_date=2022-01-15&detailed=false&api_key=Gp6t6goDrz4CeNu4wVdaHB6AIw1ugYfX1PYaKQuL', 'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2021-12-25&end_date=2022-01-01&detailed=false&api_key=Gp6t6goDrz4CeNu4wVdaHB6AIw1ugYfX1PYaKQuL', 'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2022-01-01&end_date=2022-01-08&detailed=false&api_key=Gp6t6goDrz4CeNu4wVdaHB6AIw1ugYfX1PYaKQuL'}, 'element_count': 130, 'near_earth_objects': {'2022-01-07': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2216523?api_key=Gp6t6goDrz4CeNu4wVdaHB6AIw1ugYfX1PYaKQuL'}, 'id': '2216523', 'neo_reference_id': '2216523', 'name': '216523 (2001 HY7)', 'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2216523', 'absolute_magnitude_h': 20.7, 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.1925550782, 'estimated_diameter_max': 0.4305662442}, 'meters': {'estimated_diameter_min': 

In [29]:
from src.utils import get_a_random_chunk_property
get_a_random_chunk_property(data)

date: 2022-01-12
NEO name: (2023 RF9)
neo_reference_id: 54382900


For the remaining tasks, you have to organize the data as pd.DataFrame so as to suit the specific need in each task. This part may require a considerably amount of efforts, which is normal in data science and analytics works.

In [108]:
import pandas as pd
temp_dfs = []   #Create a temp list to store all the dataframe we are going to generate

for d in data:      #The way the data is saved to a list leaves it in pieces that we have to go over
    for day in d['near_earth_objects']:
        subset = d['near_earth_objects'][day]   #We take a subset consisting of all object logged that day
        df_temp = pd.json_normalize(subset)     #Turn the subset into a dataframe
        df_temp = df_temp[['id', 'name', 'absolute_magnitude_h']]   #Retrieve the columns we want
        df_temp['date'] = day       #Save the date in a new column
        temp_dfs.append(df_temp)    #Save the dataframe in a temp list

df = pd.concat(temp_dfs, ignore_index=True)     #Combine all the saved dataframes
df

,id,name,absolute_magnitude_h,date
0,2216523,216523 (2001 HY7),20.70,2022-01-07
1,2494697,494697 (2004 SW55),20.77,2022-01-07
2,2496860,496860 (1999 XL136),19.67,2022-01-07
3,3311963,(2006 AL4),24.92,2022-01-07
4,3401388,(2008 CO),22.60,2022-01-07
...,...,...,...,...
254,54236582,(2022 AU5),26.20,2022-01-13
255,54236938,(2022 AS6),24.84,2022-01-13
256,54236939,(2022 AT6),23.23,2022-01-13
257,54240415,(2022 BH4),22.12,2022-01-13


### Task 2: Data Analysis

- Calculate the average size of the NEOs for each day.
- Determine the proportion of NEOs that are potentially hazardous.
- Find the NEO with the closest approach distance for each day.
- Use statistical methods to analyze the data. For example, calculate the mean, median, mode, and standard deviation of the NEO sizes. Determine if the size of a NEO is correlated with whether it is potentially hazardous.

In [98]:
# Write your code

### Task 3: Data Visualization Part A

- Create a line plot of the number of NEOs per week.
- Create a histogram of the distribution of NEO sizes.
- Create a bar plot of the average NEO size per week.
- Use a library like Seaborn to create more complex visualizations, such as a box plot of the NEO sizes or a heat map of the number of NEOs per week. **Be creative**!

In [77]:
# Write your code

### Task 4: Data Visualization Part B

- Create a pie chart of the proportion of hazardous vs non-hazardous NEOs.
- Create a scatter plot of the correlation between NEO size and close approach distance.
- Customize the appearance of your plots (e.g., colors, labels, titles).
- Create interactive visualizations using a library like Plotly. For example, create an interactive scatter plot where you can hover over each point to see more information about the NEO. **Be creative!**

In [99]:
# Write your code

### Task 5: Interpretation of Results

- Interpret the results of your data visualization in part A and B. 
- What insights can you gain about NEOs from your results? Summarizing your findings.
- Use your findings to make predictions or recommendations. For example, if you found that larger NEOs are more likely to be potentially hazardous, you could recommend that more resources be allocated to tracking large NEOs. **Be creative!**
- Identify, understand, and explain one scientific paper, on a clustering or classification method of relevance that could help Task 5. You don't have to implement it, you just need to justify in this notebook why the method in the scientific paper could contribute in analysis or interpretation of the results.

In [ ]:
# Write your code

### Task 6: Presentation and Documentation

- Make this project as part of your presentation, **using beamer in LaTeX**. 
- This should include an overview of your work, the results of your data analysis, and the insights you gained from your results.